In [1]:
#autoreload
%load_ext autoreload
%autoreload 2

In [2]:
from data.unlabeled import TiktokenTokenizer, unlabeledDataset
from model.gpt2 import GPT

/home/agrawalp2/miniconda3/envs/tinylm/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
model = GPT()

In [7]:
n_params = model.get_num_params()

Number of parameters: 123.74M. Number of parameters (excluding embeddings): 85.11M. Embeddings occupy 68.78% of the total parameter count. 


In [8]:
model

GPT(
  (wte): Embedding(50304, 768)
  (wpe): Embedding(1024, 768)
  (dropout): Dropout(p=0.0, inplace=False)
  (layers): ModuleList(
    (0-11): 12 x TransformerBlock(
      (layernorm1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (attn): MultiheadAttention(
        (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
      )
      (layernorm2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (mlp): MLP(
        (residual_fc): Linear(in_features=768, out_features=3072, bias=True)
        (gelu): GELU(approximate='none')
        (residual_projection): Linear(in_features=3072, out_features=768, bias=True)
        (dropout): Dropout(p=0.0, inplace=False)
      )
    )
  )
  (layernorm_final): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  (head): Linear(in_features=768, out_features=50304, bias=True)
)

In [1]:
from data.unlabeled import TiktokenTokenizer, unlabeledDataset, dataConfig
from model.gpt2 import GPT

import numpy as np
from pathlib import Path

#first write script for single GPU, vanilla training
#then write script for multi-GPU, vanilla training
#then multinode


from fastai.text.all import *

class OWTData():
    f'openwebtext data'
    def __init__(self, bin_path: str):
        self.bin_path = bin_path
    
        self.data = np.memmap(self.bin_path, dtype=np.uint16, mode='r')
    
    def __len__(self):
        #get length of data in train.bin file 
        return len(self.data)
    

data = OWTData(dataConfig().cache_dir/'train.bin')
len(data)

/home/agrawalp2/miniconda3/envs/tinylm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


FileNotFoundError: [Errno 2] No such file or directory: '~/.cache/tinyUniverse/pretraining_data/train.bin'